4.8

# Outline

1. Import

2. Finding the aggregated mean

3. Creating a Loyalty Flag

4. Finding purchase price diffrences in customers

5. Creating a Spending Amount Flag

6. Creating an Order Frequency Flag

7. Export

# Import

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
merged_df = pd.read_pickle('/Users/christinarandall/Documents/Career Foundry Projects/Achievment 4/Data/Prepared Data/merged_df.pickle')

# Finding the Aggregated Mean

In [3]:
merged_df.head(5)

,Unnamed: 0_x,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,...,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days_2,hour,order_amount
0,0,2539329,1,prior,1,2,8,NaN,196.0,1.0,...,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,8,Average Orders
1,1,2398795,1,prior,2,3,7,15.0,196.0,1.0,...,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,7,Average Orders
2,2,473747,1,prior,3,3,12,21.0,196.0,1.0,...,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,12,Most Orders
3,3,2254736,1,prior,4,4,7,29.0,196.0,1.0,...,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,7,Average Orders
4,4,431534,1,prior,5,4,15,28.0,196.0,1.0,...,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,15,Most Orders


In [4]:
merged_df.shape

(32404859, 22)

In [5]:
df_1 = merged_df

In [6]:
df_1.shape

(32404859, 22)

In [7]:
df_1 = df_1.groupby('department_id').agg({'order_number': ['mean']})

In [8]:
df_1.head(15)

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


It seems that as we increased our data sest, the average went down among the departments.

# Creating a Loyalty Flag

In [9]:
df_2 = merged_df

In [10]:
order_count = df_2.groupby(['user_id'])['order_number'].transform(max)

In [11]:
df_2.loc[order_count <= 10, 'loyalty_flag'] = 'New Customer'
df_2.loc[(order_count > 10) & (order_count <= 40), 'loyalty_flag'] = 'Regular Customer'
df_2.loc[order_count > 40, 'loyalty_flag'] = 'Loyal Customer'

In [12]:
df_2.head()

,Unnamed: 0_x,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,...,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days_2,hour,order_amount,loyalty_flag
0,0,2539329,1,prior,1,2,8,NaN,196.0,1.0,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,8,Average Orders,New Customer
1,1,2398795,1,prior,2,3,7,15.0,196.0,1.0,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,7,Average Orders,New Customer
2,2,473747,1,prior,3,3,12,21.0,196.0,1.0,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,12,Most Orders,New Customer
3,3,2254736,1,prior,4,4,7,29.0,196.0,1.0,...,77,7,9.0,both,Mid-range product,Least busy,Least busy,7,Average Orders,New Customer
4,4,431534,1,prior,5,4,15,28.0,196.0,1.0,...,77,7,9.0,both,Mid-range product,Least busy,Least busy,15,Most Orders,New Customer


#  Finding Purchase Price Diffrences in Customers

In [13]:
loyal_customer_stats = df_2.loc[df_2['loyalty_flag'] == 'Loyal Customer', 'prices'].describe()
regular_customer_stats = df_2.loc[df_2['loyalty_flag'] == 'Regular Customer', 'prices'].describe()
new_customer_stats = df_2.loc[df_2['loyalty_flag'] == 'New Customer', 'prices'].describe()

In [14]:
loyal_customer_stats

count    1.028409e+07
mean     1.038634e+01
std      3.280178e+02
min      1.000000e+00
25%      4.200000e+00
50%      7.400000e+00
75%      1.120000e+01
max      9.999900e+04
Name: prices, dtype: float64

In [15]:
regular_customer_stats

count    1.587678e+07
mean     1.249572e+01
std      5.397209e+02
min      1.000000e+00
25%      4.200000e+00
50%      7.400000e+00
75%      1.130000e+01
max      9.999900e+04
Name: prices, dtype: float64

In [16]:
new_customer_stats

count    6.243990e+06
mean     1.329467e+01
std      5.975603e+02
min      1.000000e+00
25%      4.200000e+00
50%      7.400000e+00
75%      1.130000e+01
max      9.999900e+04
Name: prices, dtype: float64

# Creating a Spending Amount Flag

In [17]:
df_3 = merged_df

In [18]:
user_mean_price = df_2.groupby('user_id')['prices'].transform(np.mean)

In [19]:
user_mean_price.head(5)

0    6.367797
1    6.367797
2    6.367797
3    6.367797
4    6.367797
Name: prices, dtype: float64

In [20]:
df_3.loc[user_mean_price < 10, 'spend_flag'] = 'Low spender'
df_3.loc[user_mean_price >= 10, 'spend_flag'] = 'High spender'

In [21]:
df_3.sample(2)

,Unnamed: 0_x,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,...,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days_2,hour,order_amount,loyalty_flag,spend_flag
14475604,3326,1041306,216,prior,4,2,21,7.0,23233.0,14.0,...,3,10.6,both,Mid-range product,Regularly busy,Regularly busy,21,Average Orders,Regular Customer,Low spender
15434770,1138218,751487,68392,prior,37,6,12,6.0,44560.0,40.0,...,19,5.6,both,Mid-range product,Regularly busy,Regularly busy,12,Most Orders,Loyal Customer,Low spender


In [22]:
df_3.shape

(32404859, 24)

# Creating an Order Frequency Flag

In [23]:
df_4 = merged_df

In [24]:
df_4['order_freq'] = df_4.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [25]:
df_4.loc[df_4['order_freq'] > 20, 'order_freq_flag'] = 'Non-frequent customer'

In [26]:
df_4.loc[(df_4['order_freq'] < 20) & (df_4['order_freq'] > 10), 'order_freq_flag'] = 'Regular customer'

In [27]:
df_4.loc[df_4['order_freq']<= 10, 'order_freq_flag'] = 'Frequent customer'

In [28]:
df_4.head()

,Unnamed: 0_x,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,...,_merge,price_range_loc,busiest_day,busiest_days_2,hour,order_amount,loyalty_flag,spend_flag,order_freq,order_freq_flag
0,0,2539329,1,prior,1,2,8,NaN,196.0,1.0,...,both,Mid-range product,Regularly busy,Regularly busy,8,Average Orders,New Customer,Low spender,20.5,Non-frequent customer
1,1,2398795,1,prior,2,3,7,15.0,196.0,1.0,...,both,Mid-range product,Regularly busy,Least busy,7,Average Orders,New Customer,Low spender,20.5,Non-frequent customer
2,2,473747,1,prior,3,3,12,21.0,196.0,1.0,...,both,Mid-range product,Regularly busy,Least busy,12,Most Orders,New Customer,Low spender,20.5,Non-frequent customer
3,3,2254736,1,prior,4,4,7,29.0,196.0,1.0,...,both,Mid-range product,Least busy,Least busy,7,Average Orders,New Customer,Low spender,20.5,Non-frequent customer
4,4,431534,1,prior,5,4,15,28.0,196.0,1.0,...,both,Mid-range product,Least busy,Least busy,15,Most Orders,New Customer,Low spender,20.5,Non-frequent customer


#  Export

In [29]:
flagged_data = df_4

In [38]:
flagged_data.to_pickle('/Users/christinarandall/Documents/Career Foundry Projects/Achievment 4/Data/Prepared Data/flagged_data.pkl')
